In [69]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.models as models
import torchvision.datasets as datasets
import numpy as np
from PIL import Image

In [70]:
class Generator(nn.Module) :
    def __init__(self, m):
        super().__init__()
        self.l1 = nn.Sequential(nn.Linear(in_features=100, out_features=256), nn.ReLU())
        self.l2 = nn.Sequential(nn.Linear(in_features=256, out_features=512), nn.ReLU())
        self.l3 = nn.Sequential(nn.Linear(in_features=512, out_features=1024), nn.ReLU())
        self.output = nn.Sequential(nn.Linear(in_features=1024, out_features=m), nn.Tanh())
    def forward(self, x) :
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.output(x)
        return x

In [ ]:
class GAN() :
    def __init__(self, n, m) :
        self.n = n
        self.m = m
        self.generator = Generator(self.m)
        self.discriminator = models.resnet50(pretrained=True)
        self.discriminator.fc = nn.Linear(in_features=512, out_features=2)
        self.discriminator = self.discriminator.to('cpu')
        self.discr_criterion = torch.nn.BCELoss()
        self.discr_optimizer = torch.optim.SGD(self.discriminator.fc.parameters(), lr=0.001, momentum=0.9)
        self.gen_criterion = torch.nn.BCELoss()
        self.gen_optimizer = torch.optim.SGD(self.discriminator.parameters(), lr=0.001, momentum=0.9)
    
    def learning_step(self, x) :
        z = torch.randn(x.shape[0], 3, self.n, 100)
        fakes = self.generator(z)
        real_classes = self.discriminator(x)
        fake_classes = self.discriminator(fakes)#find picture sizes
        fake_loss = self.discr_criterion(fake_classes, torch.ones[x.shape[0]])
        real_loss = self.discr_criterion(real_classes, torch.zeros[x.shape[0]])
        full_loss = fake_loss + real_loss
        full_loss.backward()
        self.discr_optimizer.step()
        self.generator.zero_grad()
        gen_loss = self.gen__criterion(fake_classes, torch.zeros[x.shape[0]])
        gen_loss.backward()
        self.gen_optimizer.step()
        return gen_loss, full_loss
    
    def fit(self, file_path, epochs_number) :
        trans = transforms.Compose([transforms.ToTensor(),
                                        transforms.RandomResizedCrop(224),
                                        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

        trans_data = datasets.ImageFolder(file_path, trans)
        dataloader = DataLoader(trans_data, batch_size=128, shuffle=True, num_workers=4)
        history = {'generator' : [], 'discriminator' : []}
        for epoch in range(epochs_number) :
            losses = {'generator' : [], 'discriminator' : []}
            for i, data in enumerate(dataloader, 0) :
                x, _ = data
                gen_loss, full_loss = self.learning_step(x)
                losses['generator'].append(float(gen_loss))
                losses['discriminator'].append(float(full_loss))
                history['generator'].append(float(gen_loss))
                history['discriminator'].append(float(full_loss))
            avg_gen_loss = np.mean(losses['generator'])
            avg_discr_loss = np.mean(losses['discriminator'])
            print("Epoch {} / {}: Generator Loss = {:.3f}, Discriminator Loss = {:.3f}".format(epoch+1, epochs_number, avg_gen_loss, avg_discr_loss))
        return history
    
    def predict(self, n, w, h) :
        z = torch.randn(n, 3, w, h)
        fakes = self.generator(z)
        for i, tensor in enumerate(fakes) :
            image = tensor.clone().detach().numpy()
            img = Image.fromarray(image, 'RGB')
            img.save(f'image_{i+1}.png')


In [84]:
model = GAN(100, 100)

In [85]:
model.fit('dataset', 200)

x: torch.Size([128, 3, 224, 224])
fakes: torch.Size([128, 3, 100, 100])


RuntimeError: [enforce fail at ..\c10\core\impl\alloc_cpu.cpp:72] data. DefaultCPUAllocator: not enough memory: you tried to allocate 205520896 bytes.

In [86]:
model.predict(10)